In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.dates as mdates

pd.set_option('display.max_columns', None) # pandasオプション：列データを全て表示

# データフレームdfにOur World in Dataのデータを読み込む
df = pd.read_json('https://covid.ourworldindata.org/data/owid-covid-data.json')
df = df[df.columns.drop(list(df.filter(regex='OWID')))] # OWIDデータを除外

In [ ]:
# 解析対象の国名コードを指定
country_code1 = 'FRA'
country_code2 = 'DEU'

# 国のデータをcountry1へ格納
country1 = pd.DataFrame(df[country_code1]['data'])
country1['date'] = pd.to_datetime(country1['date']) # 日付フォーマットを一括変換
country1.set_index('date',inplace = True) # dateをindexに設定

# 国のデータをcountry2へ格納
country2 = pd.DataFrame(df[country_code2]['data'])
country2['date'] = pd.to_datetime(country2['date']) # 日付フォーマットを一括変換
country2.set_index('date',inplace = True) # dateをindexに設定

In [ ]:
# 解析対象のkeyを指定
key='new_cases_smoothed'

blue = '#1f77b4'
orange = '#ff7f0e'

# keyの時系列データの表示
ax1 = country1.plot(y=key, c=blue, legend=False)
ax2 = ax1.twinx() # ax2をy2軸へ
country2.plot(y=key, ax=ax2, c=orange, legend=False)
ax1.set_ylabel(df[country_code1]['location'], color=blue)
ax2.set_ylabel(df[country_code2]['location'], color=orange)
plt.show()

In [ ]:
# keyに関して欠損値NaNを除いたデータフレームcdataを作り直す
cdata1 = country1.dropna(subset = [key])
cdata2 = country2.dropna(subset = [key])
dates1 = cdata1.index
dates2 = cdata2.index

# keyの平均値・分散値
v1_mean = cdata1[key].mean()
v1_std  = cdata1[key].std(ddof=0)
v2_mean = cdata2[key].mean()
v2_std  = cdata2[key].std(ddof=0)

# 相関の計算に必要なデータの抽出
numDates1 = len(dates1)
numDates2 = len(dates2)
maxDiffDay1 = (dates1[-1] - dates1[0]).days + 1
maxDiffDay2 = (dates2[-1] - dates2[0]).days + 1
correlSize = maxDiffDay1 + maxDiffDay2 -1
diffs = np.arange(- (maxDiffDay2 - 1), maxDiffDay1)
correlations = np.zeros([correlSize])
counts = np.zeros([correlSize])

print('country1の日数: ', numDates1)
print('country2の日数: ', numDates2)
print('country1の記録: ', dates1[0].date(), dates1[-1].date())
print('country2の記録: ', dates2[0].date(), dates2[-1].date())
print('確保する配列の要素数: ', correlSize)

# 相互相関の計算
for i_day in range(numDates1):
    for j_day in range(numDates2):
        
        # i_dayとj_dayを取り出し日の差をdiffDayに格納
        # （maxDiffDay - 1）は配列の0スタートのためのオフセット
        diffDay = (dates2[j_day] - dates1[i_day]).days + (maxDiffDay2 - 1)

        # 相互相関を計算
        correlations[diffDay] += (cdata1[key][i_day] - v1_mean)*(cdata2[key][j_day] - v2_mean)
        
        # 足した回数を記録
        counts[diffDay] += 1

# 相互相関をkey1の標準偏差*key2の標準偏差*足した回数で割る
correlations /= v1_std*v2_std*counts

print('===== 相互相関：計算完了 ===== ')

In [ ]:
# グラフを表示する範囲 [diffDayMin, diffDayMax]
diffDayMax = +200
diffDayMin = -200

# グラフを表示する範囲から最大値の場所を抜き出す
diffs = np.array(diffs)
correlations = np.array(correlations)
cliped_diffs = diffs[(diffs > diffDayMin) & (diffs < diffDayMax)]
cliped_correlations = correlations[(diffs > diffDayMin) & (diffs < diffDayMax)]
max_day    = cliped_diffs[cliped_correlations.argmax()]
max_correl = cliped_correlations[cliped_correlations.argmax()]

print('最大の相関 %d 日後：相関値 %.3f' % (max_day, max_correl))

# グラフの表示
fig, ax = plt.subplots()
ax.set_title(df[country_code2]['location'] + '-' + df[country_code1]['location'])
ax.plot(diffs, correlations)
ax.plot(max_day, max_correl, 'ro')
ax.set_xlabel('diff_day')
ax.set_ylabel('correlation')
ax.set_xlim([diffDayMin, diffDayMax])
plt.show()